# Transcoding video using OpenCV, FFMPEG, Tensorflow

In [2]:
import subprocess
import cv2 
import numpy as np
from PIL import Image
import tensorflow as tf

In [17]:
ffmpeg_cmd = ["ffmpeg",  "-y",
                "-vsync", "0",
                "-hwaccel",  "nvdec",
                "–s","1280x720", 
                "-c:v",  "h264_cuvid",
                "-i", "./Data/Source1.mp4",
                "-preset", "slow",
                "-b:v", "5M",
                "./Data/outTrans.yuv"]

In [18]:
ffmpeg1 = subprocess.Popen(ffmpeg_cmd, stdout=sp.PIPE, bufsize=10)

In [19]:
class YUV2RGB_GPU():
    def __init__(self, w=1920, h=1080): 
        config = tf.ConfigProto(gpu_options=tf.GPUOptions(per_process_gpu_memory_fraction=0.03))
        self.y = tf.placeholder(shape=(1, h, w), dtype=tf.float32)
        self.u = tf.placeholder(shape=(1, h, w), dtype=tf.float32) 
        self.v = tf.placeholder(shape=(1, h, w), dtype=tf.float32)
        r = self.y+1.371*(self.v-128)
        g = self.y+0.338* (self.u-128)-0.698*(self.v-128)
        b = self.y+1.732*(self.u-128)
        result = tf.stack([b, g, r], axis=-1)
        self.result = tf.clip_by_value(result, 0, 255)
        self.sess = tf.Session(config=config)

    def convert(self, y, u, v):
        results = self.sess.run(self.result, feed_dict={self.y:y, self.u: u, self.v: v})
        return results.astype(np.uint8)


In [21]:
if __name__ == '__main__':
    C = YUV2RGB_GPU()
    ffmpegs = [ffmpeg1]
    while True:      
        width = 640
        height = 360
        k = width*height
        
        ys = []
        us = []
        vs = []
        for i in ffmpegs:
            inpt = i.stdout.read(int(width*height*6//4))  # read bytes of single frames
            y =  np.frombuffer(inpt[0:k], dtype=np.uint8).reshape((height, width))
            u =  np.frombuffer(inpt[k:k+k//4], dtype=np.uint8).reshape((height//2, width//2))
            v =  np.frombuffer(inpt[k+k//4:], dtype=np.uint8).reshape((height//2, width//2))

            u = np.reshape(cv2.resize(np.expand_dims(u, -1), (width, height)), (height, width))
            v = np.reshape(cv2.resize(np.expand_dims(v, -1), (width, height)), (height, width))

            image = np.stack([y, u, v], axis=-1)

            ys.append(y)
            us.append(u)
            vs.append(v)

        image = C.convert(ys, us, vs)
        image = np.concatenate(image, axis=0) 
        image = cv2.resize(image, None, fx=1/2, fy=1/2)
        cv2.imshow("image", image)
        
        if cv2.waitKey(10) & 0xFF == ord('b'):
               break

    cv2.destroyAllWindows()
    print('Completed')

Completed
